# Cleanup Pipeline 4

REDO Ngram counter with learning curves.

*Purpose*
- generalize the **DocumentToBigramCounterTransformer** class to a **DocumentToNgramCounterTransformer** class. 

*Concerns*
- has same issues as Bigram class (stop words in Ngrams, no Ngram-only option)
- also noticed other transformations might not have worked, like expand_contractions (Ex. "havent_done"); why?
- is potentially slower because of the for loop in `self.n_grams`

*Results*
- using the same dataset $(m=120k,n=50k)$ on the new **DocumentToNgramCounterTransformer** class set to the default (Bigram) yields the same exact results, which is a great sign it's working the same as the **DocumentToBigramCounterTransformer** class
- using the same dataset with **Trigrams** yields slightly lower accuracies
- doubling the vocabulary to $n=100k$ yields similar accuracies
- with higher vocabularies (250, 500k) we get no improvement, but surprisingly, the training time is similar

---

In [1]:
import re
import os
import time
import json

import numpy as np
import pandas as pd

import urlextract
from html import unescape

from nltk import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import cleanup_module as Cmod

from scipy.sparse import csr_matrix
from collections import Counter

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.model_selection import train_test_split

---

### Hybrid approach

In [2]:
def expand_contractions(text, contractions_map):
    
    pattern = re.compile('({})'.format('|'.join(contractions_map.keys())), 
                        flags=re.IGNORECASE|re.DOTALL)
    
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_map.get(match)\
                                if contractions_map.get(match)\
                                else contractions_map.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def is_ascii(doc):
    try:
        doc.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [3]:
class DocumentToNgramCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, expand_contractions=True, lower_case=True, 
                 replace_usernames=True, unescape_html=True, 
                 replace_urls=True, replace_numbers=True, 
                 remove_junk=True, remove_punctuation=True, 
                 replace_emojis=True, replace_nonascii=True, 
                 remove_stopwords=True, lemmatization=True,
                 n_grams=2 # defaults to bigram
                ): 
        self.expand_contractions = expand_contractions
        self.lower_case = lower_case
        self.replace_usernames = replace_usernames
        self.unescape_html = unescape_html
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.remove_junk = remove_junk
        self.remove_punctuation = remove_punctuation
        self.replace_emojis = replace_emojis
        self.replace_nonascii = replace_nonascii
        self.remove_stopwords = remove_stopwords
        self.lemmatization = lemmatization
        self.n_grams = n_grams
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = []
        for doc in X:
            if self.lower_case:
                doc = doc.lower()
            if self.expand_contractions and contractions_map is not None:
                doc = expand_contractions(doc, contractions_map)
            if self.replace_usernames:
                doc = re.sub(r'^@([^\s]+)',' USERNAME ', doc)
            if self.unescape_html:
                doc = unescape(doc)
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(doc)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    doc = doc.replace(url, ' URL ')
            if self.replace_numbers:
                doc = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', ' NUMBER ', doc)
            if self.remove_junk:
                pattern = r'\¥|\â|\«|\»|\Ñ|\Ð|\¼|\½|\¾|\!|\?|\¿|\x82\
                            |\x83|\x84|\x85|\x86|\x87|\x88|\x89|\
                            |\x8a|\x8b|\x8c|\x8d|\x8e|\°|\µ|\´|\º|\¹|\³'
                doc = re.sub(pattern,'', doc)
            if self.remove_punctuation:
                doc = re.sub(r'\W+', ' ', doc, flags=re.M)
            if self.replace_emojis:
                doc = re.sub(r'[^\x00-\x7F]+', ' EMOJI ', doc)
            if self.replace_nonascii:
                if is_ascii(doc) == False:
                    doc = ' NONASCII '
            # tokenize
            tokens = doc.split()
            if self.remove_stopwords:
                stop_words = ['a','an','and','are','as','at','be','by','for','from',
                              'has','he','in','is','it','its','of','on','that','the',
                              'to','was','were','will','with']
                tokens = [t for t in tokens if t not in stop_words]
            if self.lemmatization and lemmatizer is not None:
                tokens = [lemmatizer.lemmatize(t) for t in tokens]
            if self.n_grams:
                for i in range(2, self.n_grams+1): # fix doubling of unigrams
                    grams = ngrams(word_tokenize(doc), i)
                    grams = ['_'.join(gram) for gram in grams]
                    tokens = [*tokens, *grams]
            # include counts
            tokens_counts = Counter(tokens)
            # append to list
            X_transformed.append(tokens_counts)
        return np.array(X_transformed)

In [4]:
with open("contractions_map.json") as f:
    contractions_map = json.load(f)

url_extractor = urlextract.URLExtract()
lemmatizer = WordNetLemmatizer()

In [5]:
corpus = ['You The love me', 
          'You do not love me',
          'You really really love food']

In [6]:
wordvec = DocumentToNgramCounterTransformer(n_grams=3)
X_trans = wordvec.fit_transform(corpus)

In [7]:
X_trans

array([Counter({'you': 1, 'love': 1, 'me': 1, 'you_the': 1, 'the_love': 1, 'love_me': 1, 'you_the_love': 1, 'the_love_me': 1}),
       Counter({'you': 1, 'do': 1, 'not': 1, 'love': 1, 'me': 1, 'you_do': 1, 'do_not': 1, 'not_love': 1, 'love_me': 1, 'you_do_not': 1, 'do_not_love': 1, 'not_love_me': 1}),
       Counter({'really': 2, 'you': 1, 'love': 1, 'food': 1, 'you_really': 1, 'really_really': 1, 'really_love': 1, 'love_food': 1, 'you_really_really': 1, 'really_really_love': 1, 'really_love_food': 1})],
      dtype=object)

In [8]:
pipe = Pipeline([('counter', DocumentToNgramCounterTransformer(n_grams=3)),
                 ('bow', Cmod.WordCounterToVectorTransformer(vocabulary_size=20)),
                 ('tfidf', TfidfTransformer())])

In [9]:
pipe_counter = pipe['counter'].fit_transform(corpus)

In [10]:
pipe_bow = pipe['bow'].fit(pipe_counter)

In [11]:
pipe_bow.vocabulary_

{'you': 1,
 'love': 2,
 'me': 3,
 'love_me': 4,
 'really': 5,
 'you_the': 6,
 'the_love': 7,
 'you_the_love': 8,
 'the_love_me': 9,
 'do': 10,
 'not': 11,
 'you_do': 12,
 'do_not': 13,
 'not_love': 14,
 'you_do_not': 15,
 'do_not_love': 16,
 'not_love_me': 17,
 'food': 18,
 'you_really': 19,
 'really_really': 20}

In [12]:
bow = pipe_bow.transform(pipe_counter)

In [13]:
bow.toarray() # first col is "words missing from vocab"

array([[0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [5, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]],
      dtype=int32)

In [14]:
 # IDF for the pipe_bow.vocabulary_
[np.around(x,3) for x in pipe['tfidf'].fit(bow).idf_[1:]]

[1.0,
 1.0,
 1.288,
 1.288,
 1.693,
 1.693,
 1.693,
 1.693,
 1.693,
 1.693,
 1.693,
 1.693,
 1.693,
 1.693,
 1.693,
 1.693,
 1.693,
 1.693,
 1.693,
 1.693]

In [15]:
tfidf = pipe['tfidf'].fit_transform(bow)

In [16]:
[np.around(x,4) for x in tfidf.toarray()]

[array([0.    , 0.2441, 0.2441, 0.3143, 0.3143, 0.    , 0.4133, 0.4133,
        0.4133, 0.4133, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    ]),
 array([0.    , 0.1881, 0.1881, 0.2423, 0.2423, 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.3186, 0.3186, 0.3186, 0.3186, 0.3186, 0.3186,
        0.3186, 0.3186, 0.    , 0.    , 0.    ]),
 array([0.8744, 0.1033, 0.1033, 0.    , 0.    , 0.3498, 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.1749, 0.1749, 0.1749])]

In [17]:
# entire pipeline produces same result but does't save IDF or vocab
end_res = pipe.fit_transform(corpus)
[np.around(x,4) for x in end_res.toarray()]

[array([0.    , 0.2441, 0.2441, 0.3143, 0.3143, 0.    , 0.4133, 0.4133,
        0.4133, 0.4133, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    ]),
 array([0.    , 0.1881, 0.1881, 0.2423, 0.2423, 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.3186, 0.3186, 0.3186, 0.3186, 0.3186, 0.3186,
        0.3186, 0.3186, 0.    , 0.    , 0.    ]),
 array([0.8744, 0.1033, 0.1033, 0.    , 0.    , 0.3498, 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.1749, 0.1749, 0.1749])]

### POC: sample 10% of the training data

About 120,000 instances.

In [18]:
# load minimally prepared X, y train subsets
raw_path = os.path.join("..","data","1_raw","sentiment140")
X_train = pd.read_csv(os.path.join(raw_path, "X_train.csv"))
y_train = pd.read_csv(os.path.join(raw_path, "y_train.csv"))

# sample 10%
X, X_rest, y, y_rest = train_test_split(X_train, y_train, test_size=0.9, random_state=42)

# create arrays
X_array = np.array(X.iloc[:, 2]).ravel()
y_array = y.iloc[:,0].ravel()

### Bigrams

Here I just confirm that the new Ngram class works just as the previous Bigram class.

In [19]:
# full pipe
pipe = Pipeline([('counter', DocumentToNgramCounterTransformer()), # should be the same as Bigram transformer
                 ('bow', Cmod.WordCounterToVectorTransformer(vocabulary_size=50000)),
                 ('tfidf', TfidfTransformer())])

In [20]:
start_time = time.time()

X_counter = pipe['counter'].fit_transform(X_array)

mins, secs = divmod(time.time() - start_time, 60)
print(f'Elapsed: {mins:0.0f} min {secs:0.0f} sec')

Elapsed: 1 min 54 sec


In [21]:
X_counter_fit = pipe['bow'].fit(X_counter)

In [22]:
for ix, tuple_ in enumerate(X_counter_fit.vocabulary_.items()):
    if ix < 10 or ix > X_counter_fit.vocabulary_size-10:
        print(tuple_)

('i', 1)
('USERNAME', 2)
('NUMBER', 3)
('my', 4)
('you', 5)
('not', 6)
('am', 7)
('have', 8)
('i_am', 9)
('me', 10)
('and_nearly', 49992)
('wishing_everyone', 49993)
('in_twitterland', 49994)
('in_december', 49995)
('mouthed', 49996)
('claw', 49997)
('ass_day', 49998)
('bagus', 49999)
('follow_gw', 50000)


In [23]:
X_bow = pipe['bow'].fit_transform(X_counter)
X_bow

<119747x50001 sparse matrix of type '<class 'numpy.intc'>'
	with 2232287 stored elements in Compressed Sparse Row format>

In [24]:
X_tfidf = pipe['tfidf'].fit_transform(X_bow)
X_tfidf

<119747x50001 sparse matrix of type '<class 'numpy.float64'>'
	with 2232287 stored elements in Compressed Sparse Row format>

#### Naive Bayes

In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

NB_clf = MultinomialNB()

# BoW with bigrams
score = cross_val_score(NB_clf, X_bow, y_array, cv=10, verbose=1, scoring='accuracy')
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy: 0.7890 (+/- 0.0022)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.9s finished


In [26]:
# Tfidf with bigrams
score = cross_val_score(NB_clf, X_tfidf, y_array, cv=10, verbose=1, scoring='accuracy')
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy: 0.7908 (+/- 0.0024)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.1s finished


#### Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression
log_clf = LogisticRegression(solver="liblinear", random_state=42)

# BoW with bigrams
score = cross_val_score(log_clf, X_bow, y_array, cv=10, verbose=1, scoring='accuracy', n_jobs=-1)
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   21.9s remaining:   14.5s


Accuracy: 0.7895 (+/- 0.0021)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   31.4s finished


In [28]:
# Tfidf with bigrams
score = cross_val_score(log_clf, X_tfidf, y_array, cv=10, verbose=1, scoring='accuracy', n_jobs=-1)
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    3.9s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    6.5s finished


Accuracy: 0.8019 (+/- 0.0024)


### Trigrams with $n=100k$

Trigrams with $n=50k$ performed just a bit worse, so I doubled the number of features.

In [29]:
pipe = Pipeline([('counter', DocumentToNgramCounterTransformer(n_grams=3)),
                 ('bow', Cmod.WordCounterToVectorTransformer(vocabulary_size=100000)), # expanded vocab for trigrams
                 ('tfidf', TfidfTransformer())])

In [30]:
start_time = time.time()

X_counter = pipe['counter'].fit_transform(X_array)

mins, secs = divmod(time.time() - start_time, 60)
print(f'Elapsed: {mins:0.0f} min {secs:0.0f} sec')

Elapsed: 2 min 37 sec


In [31]:
X_counter_fit = pipe['bow'].fit(X_counter)

In [32]:
for ix, tuple_ in enumerate(X_counter_fit.vocabulary_.items()):
    if ix < 20 or ix > X_counter_fit.vocabulary_size-20:
        print(tuple_)

('i', 1)
('USERNAME', 2)
('NUMBER', 3)
('my', 4)
('you', 5)
('not', 6)
('am', 7)
('have', 8)
('i_am', 9)
('me', 10)
('so', 11)
('but', 12)
('just', 13)
('do', 14)
('day', 15)
('USERNAME_i', 16)
('now', 17)
('this', 18)
('up', 19)
('good', 20)
('i_was_as', 99982)
('teal', 99983)
('work_day_to', 99984)
('little_boy_is', 99985)
('boy_is_sick', 99986)
('bed_i_hope', 99987)
('today_and_they', 99988)
('lord_it', 99989)
('pooped_on', 99990)
('trip_but', 99991)
('went_well_and', 99992)
('graduates_today', 99993)
('me_homesick', 99994)
('NUMBER_driving', 99995)
('and_morning', 99996)
('lol_NUMBER_NUMBER', 99997)
('is_rolling', 99998)
('coined', 99999)
('a_new_word', 100000)


In [33]:
X_bow = pipe['bow'].fit_transform(X_counter)

In [34]:
X_tfidf = pipe['tfidf'].fit_transform(X_bow)

#### Naive Bayes

In [35]:
NB_clf = MultinomialNB()

# BoW with trigrams
score = cross_val_score(NB_clf, X_bow, y_array, cv=10, verbose=1, scoring='accuracy')
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy: 0.7893 (+/- 0.0018)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.9s finished


In [36]:
# Tfidf with trigrams
score = cross_val_score(NB_clf, X_tfidf, y_array, cv=10, verbose=1, scoring='accuracy')
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy: 0.7929 (+/- 0.0018)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.2s finished


#### Logistic Regression

In [37]:
log_clf = LogisticRegression(solver="liblinear", random_state=42)

# BoW with trigrams
score = cross_val_score(log_clf, X_bow, y_array, cv=10, verbose=1, scoring='accuracy', n_jobs=-1)
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   35.9s remaining:   23.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   50.4s finished


Accuracy: 0.7913 (+/- 0.0028)


In [38]:
# Tfidf with trigrams
score = cross_val_score(log_clf, X_tfidf, y_array, cv=10, verbose=1, scoring='accuracy', n_jobs=-1)
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    6.4s remaining:    4.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    9.1s finished


Accuracy: 0.8019 (+/- 0.0019)


### Trigrams with $n=250k$

In [39]:
pipe = Pipeline([('counter', DocumentToNgramCounterTransformer(n_grams=3)),
                 ('bow', Cmod.WordCounterToVectorTransformer(vocabulary_size=250000)),
                 ('tfidf', TfidfTransformer())])

In [40]:
start_time = time.time()

X_counter = pipe['counter'].fit_transform(X_array)

mins, secs = divmod(time.time() - start_time, 60)
print(f'Elapsed: {mins:0.0f} min {secs:0.0f} sec')

Elapsed: 2 min 33 sec


In [41]:
X_counter_fit = pipe['bow'].fit(X_counter)

- Examining a few more terms:

In [42]:
for ix, tuple_ in enumerate(X_counter_fit.vocabulary_.items()):
    if ix < 20 or ix > X_counter_fit.vocabulary_size-20:
        print(tuple_)

('i', 1)
('USERNAME', 2)
('NUMBER', 3)
('my', 4)
('you', 5)
('not', 6)
('am', 7)
('have', 8)
('i_am', 9)
('me', 10)
('so', 11)
('but', 12)
('just', 13)
('do', 14)
('day', 15)
('USERNAME_i', 16)
('now', 17)
('this', 18)
('up', 19)
('good', 20)
('watching_clueless', 249982)
('clueless_to', 249983)
('and_me_makes', 249984)
('hard_lol_watching', 249985)
('lol_watching_clueless', 249986)
('watching_clueless_to', 249987)
('clueless_to_cheer', 249988)
('nordys', 249989)
('cesar', 249990)
('hour_workout', 249991)
('workout_nordys', 249992)
('nordys_eatin', 249993)
('a_salmon', 249994)
('salmon_cesar', 249995)
('cesar_salad', 249996)
('good_hour_workout', 249997)
('hour_workout_nordys', 249998)
('workout_nordys_eatin', 249999)
('nordys_eatin_a', 250000)


In [43]:
X_bow = pipe['bow'].fit_transform(X_counter)
X_tfidf = pipe['tfidf'].fit_transform(X_bow)

In [44]:
NB_clf = MultinomialNB()

# BoW with trigrams
score = cross_val_score(NB_clf, X_bow, y_array, cv=10, verbose=1, scoring='accuracy')
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy: 0.7886 (+/- 0.0019)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.4s finished


In [45]:
# Tfidf with trigrams
score = cross_val_score(NB_clf, X_tfidf, y_array, cv=10, verbose=1, scoring='accuracy')
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy: 0.7929 (+/- 0.0027)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.5s finished


In [46]:
log_clf = LogisticRegression(solver="liblinear", random_state=42)

# BoW with trigrams
score = cross_val_score(log_clf, X_bow, y_array, cv=10, verbose=1, scoring='accuracy', n_jobs=-1)
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   34.1s remaining:   22.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   54.0s finished


Accuracy: 0.7958 (+/- 0.0027)


In [47]:
# Tfidf with trigrams
score = cross_val_score(log_clf, X_tfidf, y_array, cv=10, verbose=1, scoring='accuracy', n_jobs=-1)
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    7.9s remaining:    5.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.2s finished


Accuracy: 0.8006 (+/- 0.0015)


### Trigrams with $n=500k$

In [48]:
pipe = Pipeline([('counter', DocumentToNgramCounterTransformer(n_grams=3)),
                 ('bow', Cmod.WordCounterToVectorTransformer(vocabulary_size=500000)),
                 ('tfidf', TfidfTransformer())])

In [49]:
start_time = time.time()

X_counter = pipe['counter'].fit_transform(X_array)

mins, secs = divmod(time.time() - start_time, 60)
print(f'Elapsed: {mins:0.0f} min {secs:0.0f} sec')

Elapsed: 2 min 38 sec


In [50]:
X_counter_fit = pipe['bow'].fit(X_counter)

In [51]:
for ix, tuple_ in enumerate(X_counter_fit.vocabulary_.items()):
    if ix < 15 or ix > X_counter_fit.vocabulary_size-15:
        print(tuple_)

('i', 1)
('USERNAME', 2)
('NUMBER', 3)
('my', 4)
('you', 5)
('not', 6)
('am', 7)
('have', 8)
('i_am', 9)
('me', 10)
('so', 11)
('but', 12)
('just', 13)
('do', 14)
('day', 15)
('weight_apparently', 499987)
('is_homesick_and', 499988)
('homesick_and_a', 499989)
('and_a_light', 499990)
('light_weight_apparently', 499991)
('much_cider', 499992)
('cider_pmsl', 499993)
('me_thinks_you', 499994)
('thinks_you_had', 499995)
('you_had_too', 499996)
('too_much_cider', 499997)
('much_cider_pmsl', 499998)
('ru_doing', 499999)
('hey_miley_how', 500000)


In [52]:
X_bow = pipe['bow'].fit_transform(X_counter)
X_tfidf = pipe['tfidf'].fit_transform(X_bow)

In [53]:
NB_clf = MultinomialNB()

# BoW with trigrams
score = cross_val_score(NB_clf, X_bow, y_array, cv=10, verbose=1, scoring='accuracy')
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy: 0.7887 (+/- 0.0024)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.7s finished


In [54]:
# Tfidf with trigrams
score = cross_val_score(NB_clf, X_tfidf, y_array, cv=10, verbose=1, scoring='accuracy')
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy: 0.7918 (+/- 0.0026)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.9s finished


In [55]:
log_clf = LogisticRegression(solver="liblinear", random_state=42)

# BoW with trigrams
score = cross_val_score(log_clf, X_bow, y_array, cv=10, verbose=1, scoring='accuracy', n_jobs=-1)
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   45.4s remaining:   30.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.1min finished


Accuracy: 0.7967 (+/- 0.0030)


In [56]:
# Tfidf with trigrams
score = cross_val_score(log_clf, X_tfidf, y_array, cv=10, verbose=1, scoring='accuracy', n_jobs=-1)
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    9.5s remaining:    6.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   13.2s finished


Accuracy: 0.7997 (+/- 0.0017)


In [57]:
pipe = Pipeline([('counter', DocumentToNgramCounterTransformer(n_grams=3)),
                 ('bow', Cmod.WordCounterToVectorTransformer(vocabulary_size=1000000)),
                 ('tfidf', TfidfTransformer())])

In [58]:
start_time = time.time()

X_counter = pipe['counter'].fit_transform(X_array)

mins, secs = divmod(time.time() - start_time, 60)
print(f'Elapsed: {mins:0.0f} min {secs:0.0f} sec')

Elapsed: 2 min 26 sec


In [59]:
X_counter_fit = pipe['bow'].fit(X_counter)

In [60]:
for ix, tuple_ in enumerate(X_counter_fit.vocabulary_.items()):
    if ix < 15 or ix > X_counter_fit.vocabulary_size-15:
        print(tuple_)

('i', 1)
('USERNAME', 2)
('NUMBER', 3)
('my', 4)
('you', 5)
('not', 6)
('am', 7)
('have', 8)
('i_am', 9)
('me', 10)
('so', 11)
('but', 12)
('just', 13)
('do', 14)
('day', 15)
('my_phone_wont', 999987)
('phone_wont_type', 999988)
('wont_type_so', 999989)
('type_so_i', 999990)
('can_never_tweet', 999991)
('hdc', 999992)
('footdr', 999993)
('shitful', 999994)
('hdc_footdr', 999995)
('footdr_NUMBER', 999996)
('NUMBER_shitful', 999997)
('shitful_day', 999998)
('am_worse', 999999)
('worse_hack', 1000000)


In [61]:
X_bow = pipe['bow'].fit_transform(X_counter)
X_tfidf = pipe['tfidf'].fit_transform(X_bow)

In [62]:
NB_clf = MultinomialNB()

# BoW with trigrams
score = cross_val_score(NB_clf, X_bow, y_array, cv=10, verbose=1, scoring='accuracy')
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy: 0.7880 (+/- 0.0023)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.4s finished


In [63]:
# Tfidf with trigrams
score = cross_val_score(NB_clf, X_tfidf, y_array, cv=10, verbose=1, scoring='accuracy')
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy: 0.7896 (+/- 0.0031)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.7s finished


In [64]:
log_clf = LogisticRegression(solver="liblinear", random_state=42)

# BoW with trigrams
score = cross_val_score(log_clf, X_bow, y_array, cv=10, verbose=1, scoring='accuracy', n_jobs=-1)
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   55.6s remaining:   37.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.2min finished


Accuracy: 0.7994 (+/- 0.0029)


In [65]:
# Tfidf with trigrams
score = cross_val_score(log_clf, X_tfidf, y_array, cv=10, verbose=1, scoring='accuracy', n_jobs=-1)
print(f'Accuracy: {score.mean():0.4f} (+/- {np.std(score):0.4f})')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   11.4s remaining:    7.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   15.6s finished


Accuracy: 0.7970 (+/- 0.0020)


---